In [25]:
import numpy as np
import scipy

import time
import os

from itertools import combinations, product
from functools import reduce
import copy

import pennylane as qml
from pennylane.templates.layers import RandomLayers
# from pennylane import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import matplotlib.pyplot as plt

from sklearn.datasets import make_moons

from scipy.stats import spearmanr

# import gym

In [49]:
!pip install torch torchquantum

  Using cached torch-1.13.0-cp37-cp37m-manylinux1_x86_64.whl (890.2 MB)
  Using cached torchquantum-0.1.4-py3-none-any.whl (119 kB)
  Using cached nvidia_cuda_runtime_cu11-11.7.99-py3-none-manylinux1_x86_64.whl (849 kB)
  Using cached nvidia_cudnn_cu11-8.5.0.96-2-py3-none-manylinux1_x86_64.whl (557.1 MB)
  Using cached nvidia_cublas_cu11-11.10.3.66-py3-none-manylinux1_x86_64.whl (317.1 MB)
  Using cached nvidia_cuda_nvrtc_cu11-11.7.99-2-py3-none-manylinux1_x86_64.whl (21.0 MB)
  Using cached qiskit-0.32.1-py3-none-any.whl
  Using cached tqdm-4.64.1-py2.py3-none-any.whl (78 kB)
  Using cached torchvision-0.14.0-cp37-cp37m-manylinux1_x86_64.whl (24.3 MB)
  Using cached numpy-1.21.6-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.7 MB)
  Using cached pylatexenc-2.10-py3-none-any.whl
  Using cached torchpack-0.3.1-py3-none-any.whl (34 kB)
  Using cached qiskit_ibmq_provider-0.18.1-py3-none-any.whl (237 kB)
  Using cached qiskit_terra-0.18.3-cp37-cp37m-manylinux2010_x86_64.whl

In [2]:
!pip install pennylane pennylane-qiskit qiskit matplotlib

  Using cached PennyLane-0.25.1-py3-none-any.whl (1.0 MB)
  Using cached PennyLane_qiskit-0.24.0-py3-none-any.whl (34 kB)
  Using cached qiskit-0.38.0-py3-none-any.whl
  Using cached matplotlib-3.5.3-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl (11.2 MB)
  Using cached PennyLane_Lightning-0.25.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.6 MB)
  Using cached semantic_version-2.10.0-py2.py3-none-any.whl (15 kB)
  Using cached networkx-2.6.3-py3-none-any.whl (1.9 MB)
  Using cached retworkx-0.11.0-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (1.6 MB)
  Using cached autograd-1.4-py3-none-any.whl (48 kB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl (9.6 kB)
  Using cached autoray-0.3.2-py3-none-any.whl (36 kB)
  Using cached toml-0.10.2-py2.py3-none-any.whl (16 kB)
  Using cached ninja-1.10.2.3-py2.py3-none-manylinux_2_5_x86_64.manylinux1_x86_64.whl (108 kB)
  Using cached mthree-1.0.0-cp37-cp37m-manylinux

## train rando circuits

In [ ]:
from datasets import TorchDataset
from datasets_nt import load_dataset
from create_gate_circs_np import get_circ_params, TQCirc, generate_true_random_gate_circ
from train_circ_np import train_tq_model, TQMseLoss
from create_noise_models import noisy_dev_from_backend

import pickle as pkl
import torch
import os
import numpy as np

dataset = 'vowel_2'
curr_dir = f'./random/{dataset}/'

num_qubits = 4
num_embeds = 10

num_train_steps = 1000
num_test_data = 120
num_meas_qubits = 1

param_nums = [16, 20, 24, 28, 32]

loss = TQMseLoss(num_meas_qubits, 4)

dev = qml.device('lightning.qubit', wires=4)
device = 'cpu'

train_data = TorchDataset(dataset, 'angle', 1, reshape_labels=True)
test_data = TorchDataset(dataset, 'angle', 1, False, reshape_labels=True)

train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=32, sampler=torch.utils.data.RandomSampler(train_data))
test_data_loader = torch.utils.data.DataLoader(test_data, batch_size=32, sampler=torch.utils.data.SequentialSampler(test_data))

for param_num in param_nums:
    curr_param_dir = curr_dir + f'{param_num}_params/'
    
    for i in range(25):    
        circ_dir = curr_param_dir + f'circ_{i + 1}'

        circ_gates, gate_params, inputs_bounds, weights_bounds = get_circ_params(circ_dir)
        
        losses_list = []
        accs_list = []

        for j in range(5):
            curr_train_dir = circ_dir + '/run_{}'.format(j + 1)

            if os.path.exists(curr_train_dir):
                pass
            else:
                os.mkdir(curr_train_dir)

            model = TQCirc(circ_gates, gate_params, inputs_bounds, weights_bounds, num_qubits, False).to(device)
            opt = torch.optim.SGD(model.parameters(), lr=0.05)

            curr_loss, curr_acc = train_tq_model(model, num_meas_qubits, opt, loss, train_data_loader, test_data_loader, num_test_data, num_train_steps, 100, 10)

            print(curr_loss, curr_acc)

            losses_list.append(curr_loss)
            accs_list.append(curr_acc)
            
            torch.save(model.state_dict(), curr_train_dir + '/model.pt')

        np.savetxt(circ_dir + '/val_losses.txt', losses_list)
        np.savetxt(circ_dir + '/accs.txt', accs_list)   

Step 1 | Loss: 1.125234603881836
Step 101 | Loss: 1.081327199935913
Step 201 | Loss: 1.0258609056472778
Step 301 | Loss: 1.0233039855957031
Step 401 | Loss: 0.9606773257255554
Step 501 | Loss: 0.8778469562530518
Step 601 | Loss: 0.970910906791687
Step 701 | Loss: 0.9846652746200562
Step 801 | Loss: 0.9353497624397278
Step 901 | Loss: 0.9864102602005005
1.0090720098942045 0.5416666666666666
Step 1 | Loss: 1.4256596565246582
Step 101 | Loss: 0.7811092734336853
Step 201 | Loss: 0.9821683168411255
Step 301 | Loss: 0.8622618913650513
Step 401 | Loss: 1.0451552867889404
Step 501 | Loss: 1.0148276090621948
Step 601 | Loss: 0.9255623817443848
Step 701 | Loss: 0.9688980579376221
Step 801 | Loss: 1.0088534355163574
Step 901 | Loss: 0.9666499495506287
0.9978367305082096 0.5833333333333334
Step 1 | Loss: 1.365053415298462
Step 101 | Loss: 0.9937362670898438
Step 201 | Loss: 0.9892796277999878
Step 301 | Loss: 0.9284828901290894
Step 401 | Loss: 0.9357292652130127
Step 501 | Loss: 1.069657087326049

## train human design ciurcuits

In [ ]:
x_train = np.genfromtxt('./experiment_data/fmnist_4/x_train.txt')[:, :16].reshape((len(x_train), 4, 4))
x_test = np.genfromtxt('./experiment_data/fmnist_4/x_test.txt')[:, :16].reshape((len(x_test), 4, 4))

y_train = np.genfromtxt('./experiment_data/fmnist_4/y_train.txt')
# y_train = 1 - 2 * y_train
y_test = np.genfromtxt('./experiment_data/fmnist_4/y_test.txt')
# y_test = 1 - 2 * y_test

num_params = [12, 16, 20, 24]

num_qubits = 4
num_embed_layers = 4

dev = qml.device('lightning.qubit', wires=num_qubits)

var_layer_options = [[qml.RX], [qml.RY]] * 4
enc_layer_options = [['X'], ['Y']] * 2

for curr_num_params in num_params:
    curr_dir = './human_design/angle_basic/fmnist_4/{}_params'.format(curr_num_params)
    
    if not os.path.exists(curr_dir):
        os.mkdir(curr_dir)
        
    curr_num_layers = curr_num_params // 4
    curr_weights_shape = (curr_num_layers, 1, num_qubits)
        
    for i in range(1):
        curr_circ_dir = curr_dir

        circ = generate_human_design_circ(dev, num_qubits, 'angle', 'basic', num_embed_layers, curr_num_layers, enc_layer_options, var_layer_options[:curr_num_layers], [0, 1])

#         np.savetxt(curr_circ_dir + '/layers.txt', var_layer_options[:curr_num_layers], fmt="%s")
        
        losses_list = []
        accs_list = []
        
        for j in range(5):
            curr_train_dir = curr_circ_dir + '/run_{}'.format(j + 1)

            if os.path.exists(curr_train_dir):
                pass
            else:
                os.mkdir(curr_train_dir)
                        
            info = train_qnn(circ, x_train, y_train, x_test, y_test, curr_weights_shape, 8000, 0.05, 1, mse_vec_loss, verbosity=17300, 
                                                                                            loss_window=50, init_params=None, 
                                                                                            acc_thres=1.1, shuffle=True, print_loss=50)

            val_exps = np.array([circ(x_test[i], info[-1][-1]) for i in range(len(x_test))])
            val_loss = np.array([mse_vec_loss(y_test[k], val_exps[k]) for k in range(len(x_test))]).flatten()

            acc = np.mean(np.sum(np.multiply(val_exps, y_test) > 0, 1) == 2)

            shape = np.array(info[-1]).shape
            
            np.savetxt(curr_train_dir + '/params_{}.txt'.format(j + 1), np.array(info[-1]).reshape(shape[0], np.product(shape[1:])))
            np.savetxt(curr_train_dir + '/losses_{}.txt'.format(j + 1), info[0])

            losses_list.append(val_loss)
            accs_list.append(acc)

        np.savetxt(curr_circ_dir + '/accs.txt', accs_list)
        np.savetxt(curr_circ_dir + '/val_losses.txt', losses_list) 

Step 1 | Sliding Loss Window : 1.5632922297896672
Step 51 | Sliding Loss Window : 1.2810481233434654
Step 101 | Sliding Loss Window : 1.3015128648597132


In [ ]:
!python supernet/train_search_fmnist.py --warmup_epochs 0 --steps 13725 --n_search 1 --n_qubits 4 --n_experts 1 --n_layers 6 --n_encode_layers 4 --use_angle_encoding --save "24_params" --data "./supernet/fmnist_4_data" --save_dir "./supernet/fmnist_4_angle"

2022-08-06 17:57:34.262250: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2022-08-06 17:57:34.266626: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2022-08-06 17:57:34.493074: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
(8000, 4, 4) (2000, 4, 4)
subnet: [179825, 146790, 117431, 158076, 119195, 290667], expert_idx: 0
2022-08-06 17:57:49.257991: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX512F
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-06 17:57:49.283251: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CP

## Supernet

In [15]:
def convert_supernet_circ_into_gate_circ(subnet, num_embeds, layer_rots, layer_cnots, num_qubits, angle_embed=False):
    circ_gates = []
    gate_params = []
    weights_bounds = [0]
    inputs_bounds = [0]

    curr_layers = subnet
    num_cnots = len(layer_cnots)
    
    for i in range(1):
        num_qubits = len(layer_rots[0])

        if angle_embed:
            rots = ['ry', 'rx', 'rz']
            
            for j in range(num_embeds):
                circ_gates += [rots[j % 3] for i in range(num_qubits)]
                gate_params += [[i] for i in range(num_qubits)]
                weights_bounds += [0 for i in range(num_qubits)]
                inputs_bounds += [inputs_bounds[-1] + i + 1 for i in range(2 * num_qubits - 1)]            
        else:
            for j in range(num_embeds):
                circ_gates += ['h' for i in range(num_qubits)] + ['ry' for i in range(num_qubits)] + ['cry' for i in range(num_qubits - 1)]
                gate_params += [[i] for i in range(num_qubits)] * 2 + [[i, i + 1] for i in range(num_qubits - 1)]
                weights_bounds += [0 for i in range(3 * num_qubits - 1)]
                inputs_bounds += [inputs_bounds[-1] for i in range(num_qubits)] + [inputs_bounds[-1] + i + 1 for i in range(2 * num_qubits - 1)]

        for j in range(len(curr_layers)):
            circ_gates += layer_rots[curr_layers[j] // num_cnots]
            circ_gates += ['cx' for k in layer_cnots[curr_layers[j] % num_cnots]]

            gate_params += [[k] for k in range(len(layer_rots[curr_layers[j] // num_cnots]))]
            gate_params += layer_cnots[curr_layers[j] % num_cnots]

            weights_bounds += [weights_bounds[-1] + k + 1 for k in range(num_qubits)]
            inputs_bounds += [inputs_bounds[-1] for k in range(num_qubits)]

            weights_bounds += [weights_bounds[-1] for k in range(len(layer_cnots[curr_layers[j] % num_cnots]))]
            inputs_bounds += [inputs_bounds[-1] for k in range(len(layer_cnots[curr_layers[j] % num_cnots]))]
            
    return circ_gates, gate_params, inputs_bounds, weights_bounds

In [ ]:
import pennylane as qml
import numpy as np
import os

from datasets_nt import load_dataset
from create_gate_circs import create_gate_circ
from create_noise_models import noisy_dev_from_backend

dataset = 'mnist_2'
main_dir = './supernet/mnist_2/'

x_train, y_train, x_test, y_test = load_dataset(dataset, 'supernet', 1)

# inds = np.random.choice(len(x_test), 400, False)
# x_test = x_test[inds]
# y_test = y_test[inds]

num_qubits = 4
num_cnot_configs = 4096
num_circs = 2500
num_embed_layers = 4

device_name = 'ibm_nairobi'
# dev = qml.device('lightning.qubit', wires=num_qubits)
dev = noisy_dev_from_backend(device_name, num_qubits)

param_nums = [8, 12]

for p in param_nums:
    supernet_dir = main_dir + 'search-{}_params_mb'.format(p)

    num_layers = p // num_qubits

    layer_rots = open(supernet_dir + '/rotations.txt').read().split('\n')[:-1]
    layer_rots = [''.join([j for j in i if j.isupper()]) for i in layer_rots]
    layer_rots = [[i[j * 2:j * 2 + 2].lower() for j in range(len(i) // 2)] for i in layer_rots]

    layer_cnots = open(supernet_dir + '/cnots.txt').read().split('\n')[:-1]
    layer_cnots = [''.join([j for j in i[1:] if j not in ['[', ']', ',', ' ']]) for i in layer_cnots]
    layer_cnots = [[[int(i[2 * j]), int(i[2 * j + 1])] for j in range(len(i) // 2)] for i in layer_cnots]
        
    search_space = len(layer_rots) * len(layer_cnots)
        
    accs = []
    losses = []
    circ_layers = []

    params = np.array([float(j) for j in open(supernet_dir + '/training_params.txt').read().split('\n')[-2].split(' ')]).reshape(num_layers, len(layer_rots), num_qubits)
#     params = np.genfromtxt(supernet_dir + '/training_params.txt')[-1].reshape((num_layers, search_space // num_cnot_configs, num_qubits))
        
    supernet_device_dir = supernet_dir + '/' + device_name
    
    if not os.path.exists(supernet_device_dir):
        os.mkdir(supernet_device_dir)
        
    for i in range(num_circs):
        curr_circ_desc = np.random.randint(0, search_space, num_layers)

        curr_params = np.concatenate([params[k, curr_circ_desc[k] // num_cnot_configs] for k in range(num_layers)]).flatten()

        circ_gates, gate_params, inputs_bounds, weights_bounds = convert_supernet_circ_into_gate_circ(curr_circ_desc, num_embed_layers, layer_rots, layer_cnots,
                                                                                                          num_qubits, False) 
        
        circ = create_gate_circ(dev, circ_gates, gate_params, inputs_bounds,
                                                                weights_bounds, [0], 'exp')
        
        val_exp_list = []
        
        for j in range(len(x_test)):
            val_exp_list.append(circ(x_test[j], curr_params))
            
#         val_exps = np.array(val_exp_list)
#         val_losses = np.sum(np.power(val_exps - y_test, 2), 1)
#         val_loss = np.mean(val_losses)
#         acc = np.mean(np.sum(np.multiply(val_exps, y_test) > 0, 1) == 2)

        val_exps = np.array(val_exp_list)
        val_losses = np.power(val_exps - y_test, 2)
        val_loss = np.mean(val_losses)
        acc = np.mean(val_losses < 1)
            
        print(curr_circ_desc, acc, val_loss)

        accs.append(acc)
        losses.append(val_loss)
        circ_layers.append(curr_circ_desc)
        
    np.savetxt(supernet_device_dir + '/searched_circ_layers.txt', np.array(circ_layers))
    np.savetxt(supernet_device_dir + '/searched_circ_accs.txt', accs)
    np.savetxt(supernet_device_dir + '/searched_circ_losses.txt', losses)  
#     np.savetxt(supernet_dir + '/searched_circ_accs.txt', ['{} {}'.format(circ_layers[i], accs[i]) for i in range(num_circs)], fmt="%s")

ibmqfactory.load_account:WARNING:2022-10-09 05:02:40,938: Credentials are already in use. The existing account in the session will be replaced.


[116027 146422] 0.4975 1.107759895324707
[250623 165810] 0.49625 1.0129710197448731
[192843 232735] 0.49875 1.0884611892700196
[ 23713 244328] 0.4975 1.057957830429077
[212438 313824] 0.49875 1.082640323638916
[110580  51391] 0.49625 1.0755717849731445
[  1162 304314] 0.49625 1.0253585529327394
[  9818 233045] 0.4975 1.0317518043518066
[ 16248 312547] 0.49875 1.0563563919067382
[ 52460 172118] 0.49875 1.0976454734802246
[177079 314338] 0.49625 1.0573943614959718
[179660 150562] 0.4975 1.0205478954315186
[114034 158683] 0.4975 1.0073684978485107
[245939 129108] 0.49625 1.0136721992492677
[ 58555 115904] 0.49625 1.0184808444976807
[121876 138107] 0.4975 1.0584971809387207
[259320  70916] 0.5 1.0292612075805665
[306146  69437] 0.49875 1.067581901550293
[147229 235726] 0.49875 1.0579876041412353
[186188 215792] 0.49375 1.0646990585327147
[320623 210771] 0.5 1.04738431930542
[291137   8603] 0.5 1.0522004508972167
[229027 136473] 0.49625 1.00294020652771
[175630  31886] 0.4925 1.007817630767

In [ ]:
from datasets_nt import load_dataset
from create_gate_circs import create_gate_circ
from train_circ import mse_loss, train_qnn
from create_noise_models import noisy_dev_from_backend

import pennylane as qml
import numpy as np
import os

dataset = 'mnist_2'
main_dir = './supernet/mnist_2/'

x_train, y_train, x_test, y_test = load_dataset(dataset, 'supernet', 1)

num_qubits = 4
num_cnot_configs = 4096
num_circs = 2500
num_embed_layers = 4
meas_qubits = [0]
num_steps = 8000

dev = qml.device('lightning.qubit', wires=num_qubits)

param_nums = [4, 8, 12, 16, 20]
search_nums = [100]
num_trials = [25]

# device_name = 'ibm_nairobi'
# noisy_dev = noisy_dev_from_backend(device_name, num_qubits)

for p in param_nums:
    curr_params_dir =  main_dir + 'search-{}_params_mb'.format(p)

    layer_rots = open(curr_params_dir + '/rotations.txt').read().split('\n')[:-1]
    layer_rots = [''.join([j for j in i if j.isupper()]) for i in layer_rots]
    layer_rots = [[i[j * 2:j * 2 + 2].lower() for j in range(len(i) // 2)] for i in layer_rots]

    layer_cnots = open(curr_params_dir + '/cnots.txt').read().split('\n')[:-1]
    layer_cnots = [''.join([j for j in i[1:] if j not in ['[', ']', ',', ' ']]) for i in layer_cnots]
    layer_cnots = [[[int(i[2 * j]), int(i[2 * j + 1])] for j in range(len(i) // 2)] for i in layer_cnots]
        
    circ_accs = np.genfromtxt(curr_params_dir + '/searched_circ_accs.txt')
    circ_losses = np.genfromtxt(curr_params_dir + '/searched_circ_losses.txt')
    circ_layers = np.genfromtxt(curr_params_dir + '/searched_circ_layers.txt').astype('int32').reshape((num_circs, p // num_qubits))

    for k in range(len(search_nums)):
        search_dir = curr_params_dir + '/search_{}'.format(search_nums[k])

        if not os.path.exists(search_dir):
            os.mkdir(search_dir)

        for i in range(num_trials[k]):
            curr_dir = search_dir + '/trial_{}'.format(i + 1)

            if not os.path.exists(curr_dir):
                os.mkdir(curr_dir)

            chosen_circs = range(search_nums[k] * i, search_nums[k] * i + search_nums[k])
            best_circuit_ind = chosen_circs[np.argmax(circ_accs[chosen_circs])]
            best_subnet = circ_layers[best_circuit_ind]
            print(circ_accs[best_circuit_ind], circ_losses[best_circuit_ind], best_subnet)

            np.savetxt(curr_dir + '/sampled_circuits.txt', circ_layers[chosen_circs])
            np.savetxt(curr_dir + '/sampled_circuit_accs.txt', circ_accs[chosen_circs])

            circ_gates, gate_params, inputs_bounds, weights_bounds = convert_supernet_circ_into_gate_circ(best_subnet, num_embed_layers,
                                                                                                              layer_rots, layer_cnots, num_qubits, False)

            circ = create_gate_circ(dev, circ_gates, gate_params, inputs_bounds,
                                                                weights_bounds, meas_qubits, 'exp', 'adjoint') 
            
#             noisy_circ = create_gate_circ(noisy_dev, circ_gates, gate_params, inputs_bounds,
#                                                                 weights_bounds, meas_qubits, 'exp', None)

            losses_list = []
            accs_list = []
            
            noisy_losses_list = []
            noisy_accs_list = []

            for j in range(5):
                curr_train_dir = curr_dir + '/run_{}'.format(j + 1)

                if os.path.exists(curr_train_dir):
                    pass
                else:
                    os.mkdir(curr_train_dir)


                info = train_qnn(circ, x_train, y_train, x_test, y_test, [weights_bounds[-1]], num_steps, 0.05, 1, mse_loss, verbosity=17300, 
                                                                                                    loss_window=50, init_params=None, 
                                                                                                    acc_thres=1.1, shuffle=True, print_loss=50)
                
                np.savetxt(curr_train_dir + '/params_{}.txt'.format(j + 1), info[-1])
                np.savetxt(curr_train_dir + '/losses_{}.txt'.format(j + 1), info[0])

                val_exps = [circ(x_test[i], info[-1][-1]) for i in range(len(x_test))]
                val_loss = np.array([mse_loss(y_test[k], val_exps[k]) for k in range(len(x_test))]).flatten()
                                
#                 noisy_val_exps = [noisy_circ(x_test[i], info[-1][-1]) for i in range(len(x_test))]
#                 noisy_val_loss = np.array([mse_loss(y_test[k], noisy_val_exps[k]) for k in range(len(x_test))]).flatten()

                acc = np.mean(val_loss < 1)
#                 noisy_acc = np.mean(noisy_val_loss < 1)

                losses_list.append(val_loss)
                accs_list.append(acc)
                
#                 noisy_losses_list.append(noisy_val_loss)
#                 noisy_accs_list.append(noisy_acc)

            np.savetxt(curr_dir + '/accs.txt', accs_list)
            np.savetxt(curr_dir + '/val_losses.txt', losses_list)          
            
#             np.savetxt(curr_dir + '/noisy_accs.txt', noisy_accs_list)
#             np.savetxt(curr_dir + '/noisy_val_losses.txt', noisy_losses_list)  

            np.savetxt(curr_dir + '/gates.txt', circ_gates, fmt="%s")
            np.savetxt(curr_dir + '/gate_params.txt', gate_params, fmt="%s")
            np.savetxt(curr_dir + '/inputs_bounds.txt', inputs_bounds)
            np.savetxt(curr_dir + '/weights_bounds.txt', weights_bounds)
            np.savetxt(curr_dir + '/layers.txt', best_subnet)

0.5525 1.1063296288359943 [69116]
[2022-10-13 15:33:34.542 tensorflow-2-3-cpu-py-ml-t3-xlarge-73aa47207bb156b78b2e237cb57e:10667 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2022-10-13 15:33:34.700 tensorflow-2-3-cpu-py-ml-t3-xlarge-73aa47207bb156b78b2e237cb57e:10667 INFO profiler_config_parser.py:102] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.
Step 1 | Sliding Loss Window : 1.34197263566291
Step 51 | Sliding Loss Window : 1.0670626748928778
Step 101 | Sliding Loss Window : 0.9657789974533583
Step 151 | Sliding Loss Window : 1.0654880059711835
Step 201 | Sliding Loss Window : 1.026677361020375
Step 251 | Sliding Loss Window : 1.013725929414733
Step 301 | Sliding Loss Window : 1.016212164759321
Step 351 | Sliding Loss Window : 1.0240910282547542
Step 401 | Sliding Loss Window : 1.0056956103438495
Step 451 | Sliding Loss Window : 1.0340987725757698
Step 501 | Sliding Loss Window : 0.999815902281983
Step 551 | Sliding Loss Window : 1

## train correlation circuits

In [ ]:
from create_noise_models import noisy_dev_from_backend
from datasets_nt import load_dataset
from create_gate_circs import create_gate_circ, get_circ_params
from train_circ import train_qnn, mse_loss

import numpy as np
import os

dataset = 'mnist_2'

x_train, y_train, x_test, y_test = load_dataset(dataset, 'angle', 2)

num_qubits = 4
num_embeds = 32
num_params = 12

device_name = 'ibm_nairobi'

# dev = qml.device('lightning.qubit', wires=num_qubits)
dev = noisy_dev_from_backend(device_name, num_qubits)

for i in range(800, 1000):
    curr_dir = './experiment_data/{}/trained_circuits/circ_{}'.format(dataset, i + 1)
    circ_gates, gate_params, inputs_bounds, weights_bounds = get_circ_params(curr_dir) 

    circ = create_gate_circ(dev, circ_gates, gate_params, inputs_bounds,
                                                    weights_bounds, [0], 'exp')
    
    
    noiseless_losses = np.genfromtxt(curr_dir + '/val_losses.txt')

    losses_list = []
    accs_list = []
    
    curr_dev_dir = curr_dir + '/' + device_name

#     if not os.path.exists(curr_dev_dir + '/accs_inference_only.txt'):
    if True:
        if not os.path.exists(curr_dev_dir):
            os.mkdir(curr_dev_dir)

        for j in range(5):
            curr_train_dir = curr_dir + '/run_{}'.format(j + 1)
            curr_params = np.genfromtxt(curr_train_dir + '/params_{}.txt'.format(j + 1))[-1]

            val_exps = [circ(x_test[i], curr_params) for i in range(len(x_test))]
            val_loss = np.array([mse_loss(y_test[k], val_exps[k]) for k in range(len(x_test))]).flatten()

#             acc = np.mean(np.sum(np.multiply(val_exps, y_test) > 0, 1) == 2)
            acc = np.mean(val_loss < 1)

            losses_list.append(val_loss)
            accs_list.append(acc)

        print(np.mean(noiseless_losses), np.mean(losses_list), i + 1)

        np.save(curr_dev_dir + '/val_losses_inference_only.npy', losses_list)
        np.savetxt(curr_dev_dir + '/accs_inference_only.txt', accs_list)
    else:
        print(i)

ibmqfactory.load_account:WARNING:2022-10-21 17:39:07,005: Credentials are already in use. The existing account in the session will be replaced.


In [ ]:
from create_noise_models import noisy_dev_from_backend
from datasets_nt import load_dataset
from create_human_design_circs import generate_human_design_circ
from train_circ import train_qnn, mse_vec_loss

import numpy as np
import os

dataset = 'fmnist_4'

num_qubits = 4
num_embeds = 16

meas_qubits = [0, 1]

device_name = 'ibm_nairobi'

# dev = qml.device('lightning.qubit', wires=num_qubits)
dev = noisy_dev_from_backend(device_name, num_qubits)

param_nums = [12, 16, 20, 24]

num_embed_layers = [4, 4, 1]
repetitions = [1, 1, 1]

enc_layer_options = [[['X'], ['Y']] * 2, [[1, None], [1, None]] * 2, [[None, True], [None, True]]]
var_layer_options = [[[qml.RX], [qml.RY]] * 4, [[qml.RX], [qml.RY]] * 4, [[qml.RX], [qml.RY]] * 4]
encoding_types = ['angle',  'iqp', 'amp']

for ec in range(len(encoding_types)):
    x_train, y_train, x_test, y_test = load_dataset(dataset, 'angle', repetitions[ec])
    
    x_train = x_train.reshape((len(x_train), num_embed_layers[ec], -1))
    x_test = x_test.reshape((len(x_test), num_embed_layers[ec], -1))
    
    main_dir = './human_design/{}_basic/fmnist_4'.format(encoding_types[ec])
    
    for p in param_nums:
        curr_weights_shape = (p // num_qubits, 1, num_qubits)
        
        curr_dir = main_dir + '/{}_params'.format(p)
                     
        circ = generate_human_design_circ(dev, num_qubits, encoding_types[ec], 'basic', num_embed_layers[ec], p // num_qubits, enc_layer_options[ec],
                                          var_layer_options[ec][:p // num_qubits], meas_qubits, 'exp')

        noiseless_losses = np.genfromtxt(curr_dir + '/val_losses.txt')

        losses_list = []
        accs_list = []

        curr_dev_dir = curr_dir + '/' + device_name

        if not os.path.exists(curr_dev_dir):
            os.mkdir(curr_dev_dir)

        for j in range(5):
            curr_train_dir = curr_dir + '/run_{}'.format(j + 1)
            curr_params = (np.genfromtxt(curr_train_dir + '/params_{}.txt'.format(j + 1))[-1]).reshape(curr_weights_shape)

            val_exps = [circ(x_test[i], curr_params) for i in range(len(x_test))]
            val_loss = np.array([mse_vec_loss(y_test[k], val_exps[k]) for k in range(len(x_test))]).flatten()
        
            acc = np.mean(np.sum(np.multiply(val_exps, y_test) > 0, 1) == 2)
#             acc = np.mean(val_loss < 1)

            losses_list.append(val_loss)
            accs_list.append(acc)
        
            print(np.mean(noiseless_losses[j]), np.mean(val_loss))

        print(np.mean(noiseless_losses), np.mean(losses_list))

        np.save(curr_dev_dir + '/val_losses_inference_only.npy', losses_list)
        np.savetxt(curr_dev_dir + '/accs_inference_only.txt', accs_list)

1.1453388044159414 1.174888167888662
1.120891972436198 1.15792558738042
1.1345920890283505 1.1683427165920424
1.1475126338840653 1.1759680322986752
1.1309397922193833 1.1676460975714036
1.1358550583967877 1.1689541203462404
1.0344695426299408 1.0897528717619342
1.034279321178945 1.0937643683667275
0.9943502518583685 1.058861721252442
1.075552670248871 1.1230853664547955
1.0328823173918027 1.0868842305827837
1.0343068206615855 1.0904697116837365
0.9524569381792723 1.037476434244607
0.9153394180170692 0.9974210901161069
0.9371876109701437 1.0149657557208112
0.9240627864128625 1.000400278800149
0.9868227342836751 1.059043030616001
0.9431738975726046 1.021861317899535
0.8818081008250254 0.9875618992226243
0.9525009343013034 1.0411028687833912
0.8866928566818856 0.9858564454016557
0.8593541885147974 0.9711737548517467
0.9169215650211433 1.0150859870896178
0.899455529068831 1.0001561910698071
1.5170312307115843 1.4852685138240667
1.3510008281484693 1.3618003347514818
1.4115003799443802 1.406

In [ ]:
from datasets import load_dataset
from create_gate_circs import create_gate_circ
from train_circ import train_qnn, mse_loss

dataset = 'mnist_2'

x_train, y_train, x_test, y_test = load_dataset(dataset, 'angle', 2)

num_qubits = 4
num_embeds = 32
num_params = 12

dev = qml.device('lightning.qubit', wires=num_qubits)
# dev = noisy_dev_from_backend(device_name, num_qubits)

for i in range(5):
    curr_dir = './experiment_data/mnist_2/trained_circuits/circ_{}'.format(i + 1)
    circ_gates, gate_params, inputs_bounds, weights_bounds = get_circ_params(curr_dir) 

    circ = create_gate_circ(dev, circ_gates, gate_params, inputs_bounds,
                                                    weights_bounds, [0], 'exp')    

    losses_list = []
    accs_list = []
    
    for j in range(5):
        curr_train_dir = curr_dir + '/run_{}'.format(j + 1)

        if os.path.exists(curr_train_dir):
            pass
        else:
            os.mkdir(curr_train_dir)
    
        
        info = train_qnn(circ, x_train, y_train, x_test, y_test, [num_params], 3200, 0.05, 1, mse_loss, verbosity=17300, 
                                                                                        loss_window=50, init_params=None, 
                                                                                        acc_thres=1.1, shuffle=True, print_loss=50)
        
        val_exps = [circ(x_test[i], info[-1][-1]) for i in range(len(x_test))]
        val_loss = np.array([mse_loss(y_test[k], val_exps[k]) for k in range(len(x_test))]).flatten()

        acc = np.mean(val_loss < 1)
        
        np.savetxt(curr_train_dir + '/params_{}.txt'.format(j + 1), info[-1])
        np.savetxt(curr_train_dir + '/losses_{}.txt'.format(j + 1), info[0])
        
        losses_list.append(val_loss)
        accs_list.append(acc)
        
    np.savetxt(curr_dir + '/val_losses.txt', losses_list)
    np.savetxt(curr_dir + '/accs.txt', accs_list)

Step 1 | Sliding Loss Window : 1.07049982166282
Step 51 | Sliding Loss Window : 0.7356248509023621
Step 101 | Sliding Loss Window : 0.6974548555132921
Step 151 | Sliding Loss Window : 0.7555443795244057
Step 201 | Sliding Loss Window : 0.6425067259205788
Step 251 | Sliding Loss Window : 0.627393051490073
Step 301 | Sliding Loss Window : 0.6480652473307379
Step 351 | Sliding Loss Window : 0.6527041931472806
Step 401 | Sliding Loss Window : 0.6067697105928453
Step 451 | Sliding Loss Window : 0.7652077011360546
Step 501 | Sliding Loss Window : 0.6629097579989829
Step 551 | Sliding Loss Window : 0.6791994897222828
Step 601 | Sliding Loss Window : 0.65566657280935
Step 651 | Sliding Loss Window : 0.6515732694101622
Step 701 | Sliding Loss Window : 0.6313294778703874
Step 751 | Sliding Loss Window : 0.6157393025372828
Step 801 | Sliding Loss Window : 0.5355545668608929
Step 851 | Sliding Loss Window : 0.6368686822308715
Step 901 | Sliding Loss Window : 0.5830660574091396
Step 951 | Sliding L

In [ ]:
from create_noise_models import noisy_dev_from_backend
from datasets import load_dataset
from create_gate_circs import create_gate_circ, get_circ_params
from train_circ import train_qnn, mse_vec_loss

import numpy as np
import os

dataset = 'fmnist_4'

x_train, y_train, x_test, y_test = load_dataset(dataset, 'angle', 1)

num_qubits = 4
num_embeds = 16
num_params = 18

device_name = 'ibm_nairobi'

# dev = qml.device('lightning.qubit', wires=num_qubits)
dev = noisy_dev_from_backend(device_name, num_qubits)

for i in range(800, 1000):
    curr_dir = './experiment_data/fmnist_4/trained_circuits/circ_{}'.format(i + 1)
    circ_gates, gate_params, inputs_bounds, weights_bounds = get_circ_params(curr_dir) 

    circ = create_gate_circ(dev, circ_gates, gate_params, inputs_bounds,
                                                    weights_bounds, [0, 1], 'exp')
    
    
    noiseless_losses = np.genfromtxt(curr_dir + '/val_losses.txt')

    losses_list = []
    accs_list = []
    
    curr_dev_dir = curr_dir + '/' + device_name

    if not os.path.exists(curr_dev_dir + '/accs_inference_only.txt'):   
        if not os.path.exists(curr_dev_dir):
            os.mkdir(curr_dev_dir)

        for j in range(5):
            curr_train_dir = curr_dir + '/run_{}'.format(j + 1)
            curr_params = np.genfromtxt(curr_train_dir + '/params_{}.txt'.format(j + 1))[-1]

            val_exps = [circ(x_test[i], curr_params) for i in range(len(x_test))]
            val_loss = np.array([mse_vec_loss(y_test[k], val_exps[k]) for k in range(len(x_test))]).flatten()

            acc = np.mean(np.sum(np.multiply(val_exps, y_test) > 0, 1) == 2)
    #         acc = np.mean(val_loss < 1)

            losses_list.append(val_loss)
            accs_list.append(acc)

        print(np.mean(noiseless_losses), np.mean(losses_list), i + 1)

        np.save(curr_dev_dir + '/val_losses_inference_only.npy', losses_list)
        np.savetxt(curr_dev_dir + '/accs_inference_only.txt', accs_list)
    else:
        print(i)

ibmqfactory.load_account:WARNING:2022-09-14 04:24:36,600: Credentials are already in use. The existing account in the session will be replaced.


800
801
802
803
804
805
806
807
808
809
810
811
812
813
814
815
816
817
818
819
820
821
822
823
824
825
826
827
828
829
830
831
832
833
834
835
836
837
838
839
840
841
842
843
844
845
846
847
848
849
850
851
852
853
854
855
856
857
858
859
860
861
862
863
864
865
866
867
868
869
870
871
872
873
874
875
876
877
878
879
880
1.0314339873553275 1.375657175430152 882


## noise metrics for correlaiton circuits

In [22]:
from importlib import reload

import create_gate_circs

reload(create_gate_circs)

<module 'create_gate_circs' from '/root/create_gate_circs.py'>

In [ ]:
from create_noise_models import get_real_backend_dev, noisy_dev_from_backend
from create_gate_circs import create_batched_gate_circ, get_circ_params
from datasets_nt import load_dataset
from metrics import compute_noise_metric

dataset = 'fmnist_2'

x_train, y_train, _, __ = load_dataset(dataset, 'angle', 1)

curr_dir = './ours/{}'.format(dataset)

num_qubits = 4
num_trial_params = 128
meas_qubits = [0, 1, 2, 3]

param_nums = [28, 32]

num_cdcs = 128
num_shots = 1024

device_name = 'ibmq_manila'

dev = noisy_dev_from_backend(device_name, num_qubits)
noiseless_dev = qml.device('lightning.qubit', wires=num_qubits)

score_tvds = []
actual_tvds = []

for p in param_nums:
    param_dir = curr_dir + '/{}_params'.format(p)
    
    for i in range(2500):
        circ_dir = param_dir + '/circ_{}'.format(i + 1)

        circ_gates, gate_params, inputs_bounds, weights_bounds = get_circ_params(circ_dir)

        noisy_circ = create_batched_gate_circ(dev, circ_gates, gate_params, inputs_bounds,
                                                                    weights_bounds, meas_qubits, 'probs') 

        noiseless_circ = create_batched_gate_circ(noiseless_dev, circ_gates, gate_params, inputs_bounds,
                                                                    weights_bounds, meas_qubits, 'probs') 

        noise_metric_dir = circ_dir + '/noise_metric'

        if not os.path.exists(noise_metric_dir):
            os.mkdir(noise_metric_dir)

        device_noise_metric_dir = noise_metric_dir + '/{}'.format(device_name)

        if not os.path.exists(device_noise_metric_dir):
            os.mkdir(device_noise_metric_dir)

        params = np.random.sample((num_trial_params, weights_bounds[-1])) * 2 * np.pi
        batch_data = x_train[np.random.choice(len(x_train), num_trial_params, False)]

        noiseless_res_raw = np.array(noiseless_circ(batch_data, params, shots=num_shots))
        noisy_res_raw = np.array(noisy_circ(batch_data, params, shots=num_shots))

        actual_tvd = 1 - np.mean(np.sum(0.5 * np.abs(noiseless_res_raw - noisy_res_raw), 1))

        tvd = compute_noise_metric(circ_gates, gate_params, inputs_bounds, weights_bounds, num_qubits, noiseless_dev, dev, num_cdcs=num_cdcs, num_shots=num_shots)

        np.savetxt(device_noise_metric_dir + '/metric_tvd_score.txt', [1 - tvd])
        np.savetxt(device_noise_metric_dir + '/actual_tvd_score.txt', [actual_tvd])

        score_tvds.append(1 - tvd)
        actual_tvds.append(actual_tvd)

        print(1 - tvd, actual_tvd, i)
        
    np.savetxt(param_dir + '/act_tvds.txt', actual_tvds)
    np.savetxt(param_dir + '/metric_tvds.txt', score_tvds)

ibmqfactory.load_account:WARNING:2022-11-08 09:51:04,332: Credentials are already in use. The existing account in the session will be replaced.


0.8366012573242188 0.836029052734375 0
0.8991928100585938 0.8786163330078125 1
0.9073257446289062 0.8907241821289062 2
0.93597412109375 0.9312591552734375 3
0.9108200073242188 0.9018325805664062 4
0.9016952514648438 0.8798370361328125 5
0.9038467407226562 0.8936233520507812 6
0.9011611938476562 0.872161865234375 7
0.8770904541015625 0.86029052734375 8
0.9187393188476562 0.9084854125976562 9


In [ ]:
from create_noise_models import get_real_backend_dev, noisy_dev_from_backend
from create_gate_circs import create_batched_gate_circ, get_circ_params
from datasets_nt import load_dataset
from metrics import compute_noise_metric

dataset = 'mnist_4'

x_train, y_train, _, __ = load_dataset(dataset, 'angle', 1)

curr_dir = './experiment_data/{}/trained_circuits'.format(dataset)

num_qubits = 4
num_trial_params = 128
meas_qubits = [0, 1, 2, 3]

num_cdcs = 128
num_shots = 1024

device_name = 'ibm_nairobi'

dev = noisy_dev_from_backend(device_name, num_qubits)
noiseless_dev = qml.device('lightning.qubit', wires=num_qubits)

score_tvds = []
actual_tvds = []

for i in range(1000):
    circ_dir = curr_dir + '/circ_{}'.format(i + 1)
    
    circ_gates, gate_params, inputs_bounds, weights_bounds = get_circ_params(circ_dir)

    noisy_circ = create_batched_gate_circ(dev, circ_gates, gate_params, inputs_bounds,
                                                                weights_bounds, meas_qubits, 'probs') 

    noiseless_circ = create_batched_gate_circ(noiseless_dev, circ_gates, gate_params, inputs_bounds,
                                                                weights_bounds, meas_qubits, 'probs') 
    
    noise_metric_dir = circ_dir + '/noise_metric'
    
    if not os.path.exists(noise_metric_dir):
        os.mkdir(noise_metric_dir)
    
    device_noise_metric_dir = noise_metric_dir + '/{}'.format(device_name)

    if not os.path.exists(device_noise_metric_dir):
        os.mkdir(device_noise_metric_dir)
    
    params = np.random.sample((num_trial_params, weights_bounds[-1])) * 2 * np.pi
    batch_data = x_train[np.random.choice(len(x_train), num_trial_params, False)]
    
    noiseless_res_raw = np.array(noiseless_circ(batch_data, params, shots=num_shots))
    noisy_res_raw = np.array(noisy_circ(batch_data, params, shots=num_shots))

    actual_tvd = 1 - np.mean(np.sum(0.5 * np.abs(noiseless_res_raw - noisy_res_raw), 1))
    
    tvd = compute_noise_metric(circ_gates, gate_params, inputs_bounds, weights_bounds, num_qubits, noiseless_dev, dev, num_cdcs=num_cdcs, num_shots=num_shots)
    
    np.savetxt(device_noise_metric_dir + '/metric_tvd_score.txt', [1 - tvd])
    np.savetxt(device_noise_metric_dir + '/actual_tvd_score.txt', [actual_tvd])
    
    score_tvds.append(1 - tvd)
    actual_tvds.append(actual_tvd)
    
    print(1 - tvd, actual_tvd, i)

# np.savetxt(curr_dir + '/noise_scores.txt', score_tvds)
# np.savetxt(curr_dir + '/noisy_fids.txt', actual_tvds)

ibmqfactory.load_account:WARNING:2022-11-05 17:18:48,593: Credentials are already in use. The existing account in the session will be replaced.


0.8216400146484375 0.8183135986328125 0
0.8257064819335938 0.8568038940429688 1
0.8542327880859375 0.8488998413085938 2
0.5913772583007812 0.6678237915039062 3
0.8968582153320312 0.90057373046875 4
0.8803024291992188 0.879608154296875 5
0.8677444458007812 0.8799591064453125 6
0.828094482421875 0.8404617309570312 7
0.8075180053710938 0.8140945434570312 8
0.8330917358398438 0.735107421875 9
0.8752288818359375 0.8941268920898438 10
0.8305511474609375 0.8450546264648438 11
0.8509674072265625 0.8371658325195312 12
0.8288421630859375 0.8446502685546875 13
0.709869384765625 0.7684860229492188 14


## train correlation circuits

In [ ]:
from datasets import TorchDataset
from create_gate_circs_np import get_circ_params, TQCirc, generate_true_random_gate_circ
from train_circ_np import train_tq_model, TQMseLoss

import pickle as pkl
import torch
import os
import numpy as np

dataset = 'vowel_4'
curr_dir = f'./experiment_data/{dataset}/trained_circuits/'

num_qubits = 4
num_embeds = 10

num_train_steps = 1000
num_test_data = 120

num_meas_qubits = 2

loss = TQMseLoss(num_meas_qubits, 4)

dev = qml.device('lightning.qubit', wires=4)
device = 'cpu'

train_data = TorchDataset(dataset, 'angle', 1, reshape_labels=False)
test_data = TorchDataset(dataset, 'angle', 1, False, reshape_labels=False)

train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=32, sampler=torch.utils.data.RandomSampler(train_data))
test_data_loader = torch.utils.data.DataLoader(test_data, batch_size=32, sampler=torch.utils.data.SequentialSampler(test_data))

for i in range(600, 800):    
    circ_dir = curr_dir + f'circ_{i + 1}'
    
    circ_gates, gate_params, inputs_bounds, weights_bounds = get_circ_params(circ_dir)

    losses_list = []
    accs_list = []
    
    for j in range(5):
        curr_train_dir = circ_dir + '/run_{}'.format(j + 1)
        
        if os.path.exists(curr_train_dir):
            pass
        else:
            os.mkdir(curr_train_dir)
    
        model = TQCirc(circ_gates, gate_params, inputs_bounds, weights_bounds, num_qubits, False).to(device)
        opt = torch.optim.SGD(model.parameters(), lr=0.05)
    
        curr_loss, curr_acc = train_tq_model(model, num_meas_qubits, opt, loss, train_data_loader, test_data_loader, num_test_data, num_train_steps, 100, 10)
        
        print(curr_loss, curr_acc)
        
        torch.save(model.state_dict(), curr_train_dir + '/model.pt')

        losses_list.append(curr_loss)
        accs_list.append(curr_acc)
        
    np.savetxt(circ_dir + '/val_losses.txt', losses_list)
    np.savetxt(circ_dir + '/accs.txt', accs_list)   

Step 1 | Loss: 2.1082446575164795
Step 101 | Loss: 1.881377935409546
Step 201 | Loss: 1.7412676811218262
Step 301 | Loss: 1.7231950759887695
Step 401 | Loss: 1.9150676727294922
Step 501 | Loss: 1.5588871240615845
Step 601 | Loss: 1.9224148988723755
Step 701 | Loss: 1.6747769117355347
Step 801 | Loss: 1.4736231565475464
Step 901 | Loss: 1.8074861764907837
1.6518270992950495 0.4083333333333333
Step 1 | Loss: 2.5762627124786377
Step 101 | Loss: 1.8290566205978394
Step 201 | Loss: 1.7432793378829956
Step 301 | Loss: 1.6104679107666016
Step 401 | Loss: 1.7214608192443848
Step 501 | Loss: 1.5585615634918213
Step 601 | Loss: 1.6107914447784424
Step 701 | Loss: 1.6413596868515015
Step 801 | Loss: 1.4628678560256958
Step 901 | Loss: 1.5547655820846558
1.369958222202173 0.5166666666666667
Step 1 | Loss: 1.9465932846069336
Step 101 | Loss: 1.8343799114227295
Step 201 | Loss: 1.8306108713150024
Step 301 | Loss: 1.6922720670700073
Step 401 | Loss: 1.8281623125076294
Step 501 | Loss: 1.5725090503692